In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
%matplotlib inline
from tensorflow.keras.preprocessing import image
import random
import pandas as pd

In [ ]:
import pickle
pickle_file_path = 'labeled_data.pkl'

# Load the pickle file
with open(pickle_file_path, 'rb') as f:
    labeled_data = pickle.load(f)

In [ ]:
import pickle
pickle_file_path = 'filtered_df.pkl'

# Load the pickle file
with open(pickle_file_path, 'rb') as f:
    filtered_df = pickle.load(f)

In [ ]:
filtered_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg


In [ ]:
import random

ind = filtered_df.index.tolist()
random.shuffle(ind)

In [ ]:
n = len(filtered_df)
p_train = 0.6
p_val = 0.2
n_train = int(p_train*n)
n_val = int(p_val*n)
train_ind = ind[:n_train]
val_ind = ind[n_train:(n_train+n_val)]
test_ind = ind[(n_train+n_val):]

In [ ]:
train_img = []
val_img = []
test_img = []
train_label = []
val_label = []
test_label = []
test_ids = []

for img in labeled_data:
    if img['index'] in train_ind:
        train_img.append(img['img'])
        train_label.append(img['label'])
    elif img['index'] in val_ind:
        val_img.append(img['img'])
        val_label.append(img['label'])
    elif img['index'] in test_ind:
        test_img.append(img['img'])
        test_label.append(img['label'])
        test_ids.append(img['index'])

In [ ]:
from datasets import Dataset

train_ds = Dataset.from_dict({'img':train_img,'label':train_label})
val_ds = Dataset.from_dict({'img':val_img,'label':val_label})
test_ds = Dataset.from_dict({'img':test_img,'label':test_label})

In [ ]:
from transformers import ViTFeatureExtractor

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
def preprocess_images(examples):
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']

    return examples

In [ ]:
top_labels = pd.DataFrame(filtered_df.groupby('articleType').size().reset_index().sort_values(0,ascending = False)[:11]['articleType'])
top_labels_list = sorted(list(top_labels['articleType']))
top_labels['label_num'] = top_labels['articleType'].apply(lambda x: top_labels_list.index(x))
top_labels

,articleType,label_num
7,Tshirts,7
4,Shirts,4
0,Casual Shoes,0
9,Watches,9
5,Sports Shoes,5
3,Kurtas,3
6,Tops,6
1,Handbags,1
2,Heels,2
8,Wallets,8


In [ ]:
from datasets import Features, ClassLabel, Array3D

# we need to define the features ourselves as both the img and pixel_values have a 3D shape 
features = Features({
    'label': ClassLabel(names = top_labels_list),
    'img': Array3D(dtype="int64", shape=(3,32,32)),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
})

preprocessed_train_ds = train_ds.map(preprocess_images, batched=True, features=features)
preprocessed_val_ds = val_ds.map(preprocess_images, batched=True, features=features)
preprocessed_test_ds = test_ds.map(preprocess_images, batched=True, features=features)

Map:   0%|          | 0/1610 [00:00<?, ? examples/s]

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

Map:   0%|          | 0/531 [00:00<?, ? examples/s]

In [ ]:
preprocessed_train_ds[0].keys()

dict_keys(['label', 'img', 'pixel_values'])

In [ ]:
import tensorflow as tf

def to_tf_dataset(hf_dataset, batch_size=8):
    """Convert HuggingFace dataset to TensorFlow dataset
    
    Args:
        hf_dataset: HuggingFace dataset to convert
        batch_size: Batch size for the dataset
    """
    def generator():
        for example in hf_dataset:
            yield (
                example['pixel_values'],  # Image data
                example['label']  # Label
            )
    
    # Create TensorFlow dataset
    tf_dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(3, 224, 224), dtype=tf.float32),  # Image shape
            tf.TensorSpec(shape=(), dtype=tf.int64)  # Label shape
        )
    )
    
    # Transform the dataset
    tf_dataset = tf_dataset.map(
        lambda x, y: (
            tf.transpose(x, [1, 2, 0]),  # Transpose from (C,H,W) to (H,W,C)
            tf.one_hot(y, depth=10)  # Convert label to one-hot
        )
    )
    
    # Batch and prefetch
    return tf_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Convert datasets
print("Converting training dataset...")
train_tf_ds = to_tf_dataset(preprocessed_train_ds)
print("Converting validation dataset...")
val_tf_ds = to_tf_dataset(preprocessed_val_ds)
print("Converting test dataset...")
test_tf_ds = to_tf_dataset(preprocessed_test_ds)

# Verify the dataset format
for images, labels in train_tf_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Labels batch shape:", labels.shape)

Converting training dataset...
Converting validation dataset...
Converting test dataset...
Image batch shape: (8, 224, 224, 3)
Labels batch shape: (8, 10)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Load ResNet50 without the top layer
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers (optional)
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
output = Dense(10, activation="softmax")(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


# Train the model
history = model.fit(
    train_tf_ds,
    validation_data=val_tf_ds,
    epochs=4
)


Epoch 1/4
204/204 ━━━━━━━━━━━━━━━━━━━━ 66s 306ms/step - accuracy: 0.8099 - loss: 0.4918 - val_accuracy: 0.7377 - val_loss: 0.8021
Epoch 2/4
204/204 ━━━━━━━━━━━━━━━━━━━━ 86s 421ms/step - accuracy: 0.8156 - loss: 0.4676 - val_accuracy: 0.7358 - val_loss: 0.8060
Epoch 3/4
204/204 ━━━━━━━━━━━━━━━━━━━━ 74s 363ms/step - accuracy: 0.8188 - loss: 0.4636 - val_accuracy: 0.7321 - val_loss: 0.8108
Epoch 4/4
204/204 ━━━━━━━━━━━━━━━━━━━━ 78s 382ms/step - accuracy: 0.8228 - loss: 0.4552 - val_accuracy: 0.7321 - val_loss: 0.8079


In [ ]:
test_loss, test_acc = model.evaluate(test_tf_ds)
print(f"Test Accuracy: {test_acc:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 21s 324ms/step - accuracy: 0.7355 - loss: 0.7991
Test Accuracy: 0.7481


In [ ]:
model.save(r'C:\Users\ASUS\Desktop\shoppin-assignment\resnet50-trained\model.h5')

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import load_model
model = load_model(r'C:\Users\ASUS\Desktop\shoppin-assignment\resnet50-trained\model.h5')

In [ ]:
def predict_with_resnet50(model, test_dataset):
    """
    Perform predictions using ResNet50.
    """
    embeddings = []
    predictions = []
    
    # Print all layer names to help identify the correct layer
    print("\nAvailable layers:")
    for i, layer in enumerate(model.layers):
        print(f"{i}: {layer.name}")
    
    # Create embedding model using the second-to-last layer
    # You can adjust the index based on the printed layer names
    embedding_layer = model.layers[-2]  # Get second-to-last layer
    print(f"\nUsing layer '{embedding_layer.name}' for embeddings")
    
    embedding_model = tf.keras.Model(
        inputs=model.input,
        outputs=embedding_layer.output
    )
    
    # Iterate through the dataset
    for images, labels in tqdm(test_dataset, desc="Generating predictions"):
        try:
            # Get predictions and embeddings
            preds = model.predict(images, verbose=0)
            embs = embedding_model.predict(images, verbose=0)
            
            # Store results
            embeddings.append(embs)
            predictions.append(preds)
        except Exception as e:
            print(f"Error processing batch: {str(e)}")
            continue
    
    # Concatenate all batches
    embeddings = np.concatenate(embeddings, axis=0)
    predictions = np.concatenate(predictions, axis=0)
    
    print(f"\nFinal shapes:")
    print(f"Embeddings shape: {embeddings.shape}")
    print(f"Predictions shape: {predictions.shape}")
    
    return embeddings, predictions

# Use the function with the TensorFlow dataset
print("Generating embeddings and predictions...")
embeddings, predictions = predict_with_resnet50(model, test_tf_ds)

# Create vectors dictionary with test IDs
vectors = {test_ids[i]: embeddings[i] for i in range(len(test_ids))}

Generating embeddings and predictions...

Available layers:
0: input_layer_1
1: conv1_pad
2: conv1_conv
3: conv1_bn
4: conv1_relu
5: pool1_pad
6: pool1_pool
7: conv2_block1_1_conv
8: conv2_block1_1_bn
9: conv2_block1_1_relu
10: conv2_block1_2_conv
11: conv2_block1_2_bn
12: conv2_block1_2_relu
13: conv2_block1_0_conv
14: conv2_block1_3_conv
15: conv2_block1_0_bn
16: conv2_block1_3_bn
17: conv2_block1_add
18: conv2_block1_out
19: conv2_block2_1_conv
20: conv2_block2_1_bn
21: conv2_block2_1_relu
22: conv2_block2_2_conv
23: conv2_block2_2_bn
24: conv2_block2_2_relu
25: conv2_block2_3_conv
26: conv2_block2_3_bn
27: conv2_block2_add
28: conv2_block2_out
29: conv2_block3_1_conv
30: conv2_block3_1_bn
31: conv2_block3_1_relu
32: conv2_block3_2_conv
33: conv2_block3_2_bn
34: conv2_block3_2_relu
35: conv2_block3_3_conv
36: conv2_block3_3_bn
37: conv2_block3_add
38: conv2_block3_out
39: conv3_block1_1_conv
40: conv3_block1_1_bn
41: conv3_block1_1_relu
42: conv3_block1_2_conv
43: conv3_block1_2_bn


Generating predictions: 65it [00:28,  2.25it/s]


Final shapes:
Embeddings shape: (520, 256)
Predictions shape: (520, 10)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def compute_similarity_matrix(embeddings):
    """
    Compute a pairwise cosine similarity matrix from the embeddings.
    """
    embeddings = np.array(embeddings)
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix

def evaluate_similarity(vectors, labels, top_k=5):
    """
    Evaluate the model using embeddings for image similarity.
    
    Args:
        vectors (dict): A dictionary of image IDs and their corresponding embeddings.
        labels (list): Ground truth labels for the images.
        top_k (int): Number of top results to consider for evaluation.

    Returns:
        dict: Dictionary of evaluation metrics.
    """
    # Convert vectors to matrix and maintain IDs
    ids = list(vectors.keys())  # Convert keys to list to ensure consistent ordering
    embeddings = [vectors[key] for key in ids]  # Use ids list directly
    
    # Convert embeddings to 2D array if needed
    embeddings = np.array(embeddings)
    if len(embeddings.shape) == 3:
        embeddings = embeddings.squeeze(1)  # Remove extra dimension if present
        
    similarity_matrix = compute_similarity_matrix(embeddings)

    # Evaluation metrics
    correct_top_k = 0
    total_queries = len(labels)
    
    # Convert labels to numpy array for easier indexing
    labels = np.array(labels)
    
    for idx in tqdm(range(len(ids)), desc="Evaluating Precision@K"):
        query_label = labels[idx]
        
        # Get similarities for current query
        similarities = similarity_matrix[idx].copy()  # Make a copy to avoid modifying original
        similarities[idx] = -np.inf  # Exclude self-match
        
        # Get top-k most similar image indices
        top_k_indices = np.argsort(similarities)[-top_k:]
        
        # Check if any of the top-k have the same label
        top_k_labels = labels[top_k_indices]
        if query_label in top_k_labels:
            correct_top_k += 1

    precision_at_k = correct_top_k / total_queries

    return {
        "precision@K": precision_at_k,
        "total_queries": total_queries,
        "correct_top_k": correct_top_k
    }


In [ ]:
# Run evaluation
results = evaluate_similarity(vectors, test_label, top_k=1)
print("Topk=1 Evaluation Results:", results)

print('---------------------------------------------------------')

results = evaluate_similarity(vectors, test_label, top_k=5)
print("Topk=5 Evaluation Results:", results)

print('---------------------------------------------------------')

results = evaluate_similarity(vectors, test_label, top_k=10)
print("Topk=10 Evaluation Results:", results)

Evaluating Precision@K: 100%|██████████| 520/520 [00:00<00:00, 29409.11it/s]


Topk=1 Evaluation Results: {'precision@K': 0.7653846153846153, 'total_queries': 520, 'correct_top_k': 398}
---------------------------------------------------------


Evaluating Precision@K: 100%|██████████| 520/520 [00:00<00:00, 44097.01it/s]


Topk=5 Evaluation Results: {'precision@K': 0.9442307692307692, 'total_queries': 520, 'correct_top_k': 491}
---------------------------------------------------------


Evaluating Precision@K: 100%|██████████| 520/520 [00:00<00:00, 121249.62it/s]

Topk=10 Evaluation Results: {'precision@K': 0.9769230769230769, 'total_queries': 520, 'correct_top_k': 508}
